In [4]:
from bs4 import BeautifulSoup
import requests 

In [5]:
def linkText(link):
    url = "{}".format(link)
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"lxml")
    article_text = ''
    mylink = soup.find('div').findAll('p')
    for i in mylink:
        article_text += ''.join(i.findAll(text = True))
    return article_text

In [6]:
def YahooFetch():
    url = "https://news.yahoo.com/rss/"
    url = requests.get(url)
    soup = BeautifulSoup(url.text,'xml')
    
    try:
        # Fetch All Required items

        titles = soup.findAll('title')
        links = soup.findAll('link')
        pubDates = soup.findAll('pubDate')

    except Exception as e :
        # Return Empty if titles, links, descriptions, pubdates not found
       
        print(e)
        return "Error"

    # We don't want the first elements as these are just metadata
    
    titles.pop(0)
    titles.pop(0)
    links.pop(0)
    links.pop(0)
    pubDates.pop(0)

    #Dictionary to hold crawled information

    FeedDict = {}
    temp = []
    temptext = []
    try:
        for i in titles:
            temp.append(i.get_text())
            FeedDict['title'] = temp
        temp = []
        for i in links:
            temp.append(i.get_text())
            temptext.append(linkText(i.get_text()))
            FeedDict['link'] = temp
            FeedDict['linktext'] = temptext
        temp = []
        for i in pubDates:
            temp.append(i.get_text())
            FeedDict['pubdate'] = temp
        
        print("Success")
    except Exception as e :
        print(e)
        return "Error"
    
    return FeedDict


In [7]:
import time
start_time = time.time()
FeedDict = YahooFetch()
print("--- %s seconds ---" % (time.time() - start_time))

Success
--- 120.22221279144287 seconds ---


In [32]:
# print(FeedDict['title'][0])
# print(FeedDict['link'][0])
# print(FeedDict['linktext'][0])
# print(FeedDict['pubdate'][0])

In [17]:
import re
docs = []
for i in range(0,len(FeedDict['title'])):
    text = re.sub('&#39','',FeedDict['title'][i])
    docs.append(text)

In [18]:
docs

[';You embarrassed yourself;: Kellyanne Conway blasts CNN;s Wolf Blitzer for playing George Conway clip',
 'Virginia police say wanted Marine deserter sought family',
 'Man caught at Houston airport with 35 pounds of liquid cocaine in shampoo bottles',
 'Tempers flare over rebuilding of Notre-Dame spire',
 'Fox Regular Claims George Soros ‘Controls a Very Large Part’ of the State Department',
 'Russia blames fatal plane crash on pilots, including one who lied to get license',
 'Supreme Court Justice Brett Kavanaugh gets hero;s welcome from conservative Federalist Society',
 'Pirates attacked an Italian ship off the coast of Mexico — the latest sign of a growing criminal industry',
 'DNC Announces 10 Candidates in Atlanta Democratic Debate',
 'Trump: Impeachment ;has been very hard on my family;',
 'American war veteran who spent several days in ICE detention receives $190k settlement',
 'The Latest: Officer says Miranda failure was a mistake',
 'Vietnam jails music teacher for ;undermi

In [19]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in docs] 

In [20]:
doc_clean

[['you',
  'embarrassed',
  'yourself',
  'kellyanne',
  'conway',
  'blast',
  'cnns',
  'wolf',
  'blitzer',
  'playing',
  'george',
  'conway',
  'clip'],
 ['virginia',
  'police',
  'say',
  'wanted',
  'marine',
  'deserter',
  'sought',
  'family'],
 ['man',
  'caught',
  'houston',
  'airport',
  '35',
  'pound',
  'liquid',
  'cocaine',
  'shampoo',
  'bottle'],
 ['temper', 'flare', 'rebuilding', 'notredame', 'spire'],
 ['fox',
  'regular',
  'claim',
  'george',
  'soros',
  '‘controls',
  'large',
  'part’',
  'state',
  'department'],
 ['russia',
  'blame',
  'fatal',
  'plane',
  'crash',
  'pilot',
  'including',
  'one',
  'lied',
  'get',
  'license'],
 ['supreme',
  'court',
  'justice',
  'brett',
  'kavanaugh',
  'get',
  'hero',
  'welcome',
  'conservative',
  'federalist',
  'society'],
 ['pirate',
  'attacked',
  'italian',
  'ship',
  'coast',
  'mexico',
  '—',
  'latest',
  'sign',
  'growing',
  'criminal',
  'industry'],
 ['dnc', 'announces', '10', 'candidat

In [21]:
import gensim
from gensim import corpora

In [25]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [26]:
Lda = gensim.models.ldamodel.LdaModel

In [27]:
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [31]:
topics = ldamodel.print_topics(num_topics=3)
for topic in topics:
    print(topic)

(0, '0.010*"get" + 0.010*"russia" + 0.010*"big" + 0.010*"change" + 0.010*"u" + 0.010*"2020" + 0.010*"photo" + 0.006*"shooting" + 0.006*"day" + 0.005*"american"')
(1, '0.020*"trump" + 0.012*"leader" + 0.009*"george" + 0.009*"yovanovitch" + 0.009*"fox" + 0.009*"impeachment" + 0.005*"state" + 0.005*"say" + 0.005*"call" + 0.005*"police"')
(2, '0.010*"school" + 0.007*"shooting" + 0.007*"family" + 0.007*"khashoggi’s" + 0.007*"conway" + 0.007*"—" + 0.007*"santa" + 0.007*"murder" + 0.007*"california" + 0.007*"teacher"')


In [41]:
new_docs = [clean(doc).split() for doc in new_doc] 

In [42]:
# new_doc = "American teacher's death is being investigated Thursday as a murder"
# new_doc = clean(new_doc)
new_doc_bow = [dictionary.doc2bow(doc) for doc in new_docs]
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[[], [], [], [], [], [], [], [(335, 1)], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [(335, 1)], [], [], [], [], [], [], [], [], [], [], [], [], [], [(213, 1)], [], [], [], [], [], [], [], [(213, 1)], [], [], [], []]
